# Termperature

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
temperature = pd.read_csv(
    "../data/limited/temperature.csv",
    parse_dates=True,
    index_col="Dátum"
)
temperature.index.freq = "B"

## Plot

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(temperature)
plt.title("Budapesti Értéktőzsde Indexe 2015-2019")
plt.xlabel("Év")
plt.ylabel("temperature Index")
plt.grid(True)
plt.savefig("../docs/figures/temperature.png")
plt.savefig("../docs/figures/temperature.svg")
plt.show()

## Split dataset

In [ ]:
temperature_train = temperature.loc[:"2018"]
temperature_test = temperature.loc["2019":]
temperature_train.index.freq = "B"
temperature_test.index.freq = "B"

## SARIMA

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import MSTL

# 1. Step: Create MSTL model with weekly and yearly periodicity
mstl = MSTL(temperature_train["Utolsó ár"], periods=(21))
mstl_result = mstl.fit()
fig = mstl_result.plot()
fig.set_size_inches(15, 10)
plt.tight_layout()
plt.show()


# Plot ACF and PACF
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,10))
plot_acf(mstl_result.resid, lags=365, zero=False, ax=ax1)
plot_pacf(mstl_result.resid, lags=365, zero=False, ax=ax2)
plt.show()

# 2. Step: plot residuals ARIMA
resid_arima = ARIMA(mstl_result.resid, order=(1, 0, 1)).fit()
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,10))
plot_acf(resid_arima.resid, lags=25, zero=False, ax=ax1)
plot_pacf(resid_arima.resid, lags=25, zero=False, ax=ax2)
plt.show()


# Forecasting
temperature_forecast = pd.DataFrame(
    index=temperature_test.index,
    columns=temperature_test.columns
)
for i, date in enumerate(temperature_test.index):
    # Create MSTL model with weekly and yearly periodicity
    mstl = MSTL(temperature_train["Utolsó ár"], periods=(21))
    mstl_result = mstl.fit()

    # Decomposition
    trend = mstl_result.trend
    seasonal_021 = mstl_result.seasonal
    resid = mstl_result.resid

    resid_arima = ARIMA(resid, order=(1, 0, 1)).fit()

    # Forecasting
    trend_forecast = trend.iloc[-1]
    seasonal_021_forecast = seasonal_021.iloc[-(21 - (i % 21))] if i % 21 != 0 else seasonal_021.iloc[-21]

    seasonal_forecast = seasonal_021_forecast
    resid_forecast = resid_arima.forecast(steps=1).iloc[0]

    forecast = trend_forecast + seasonal_forecast + resid_forecast

    temperature_forecast.loc[date, "Utolsó ár"] = forecast

    actual_value = temperature_test.loc[date, "Utolsó ár"]

    new_row = pd.DataFrame(
        {"Utolsó ár": [actual_value]}, 
        index=[date]
    )
    new_row.index.freq = "D"
    temperature_train = pd.concat([temperature_train, new_row])

In [ ]:
# Plot the result
plt.figure(figsize=(12, 5))

# Plot actual data (differences)
plt.plot(
    temperature_test.index,
    temperature_test["Utolsó ár"], 
    label="temperature - Tényadat",
    color="blue",
    linewidth=1,
)

# Plot forecasted data
plt.plot(
    temperature_forecast.index,
    temperature_forecast["Utolsó ár"],
    label="temperature - Előrejelzett érték",
    color="green",
    linewidth=1,
)

plt.title("temperature index valós és előrejelzett alakulása 2019-ben")
plt.xlabel("Év")
plt.ylabel("temperature index")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../docs/figures/temperature_daily_forecast.png")
plt.savefig("../docs/figures/temperature_daily_forecast.svg")
plt.show()